<a href="https://colab.research.google.com/github/harsh194/machine_learning/blob/main/Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os
new_directory = '/content/drive/MyDrive/Harsh/Unsupervised Learning/Week 2/Collaborative Filtering'
os.chdir(new_directory)

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from recsys_utils import *

In [11]:
X, W, b, num_movies, num_features, num_users = load_precalc_params_small()
Y,R = load_ratings_small()

print("Y", Y.shape, "R", R.shape)
print("X", X.shape)
print("W", W.shape)
print("b", b.shape)
print("num_features", num_features)
print("num_movies",   num_movies)
print("num_users",    num_users)

Y (4778, 443) R (4778, 443)
X (4778, 10)
W (443, 10)
b (1, 443)
num_features 10
num_movies 4778
num_users 443


In [12]:
tsmean = np.mean(Y[0, R[0,:].astype(bool)])
print(f"Average rating for movie 1 : {tsmean : 0.3f} / 5")

Average rating for movie 1 :  3.400 / 5


In [13]:
def cofi_cost_func(X, W, b, Y, R, lambda_):
    nm, nu = Y.shape
    J = 0

    for j in range(nu):
        w = W[j,:]
        b_j = b[0,j]
        for i in range(nm):
            x = X[i,:]
            y = Y[i,j]
            r = R[i,j]
            J += r * np.square((np.dot(w,x) + b_j - y))

    J += (lambda_)*(np.sum(np.square(W)) + np.sum(np.square(X)))
    J = J/2

    return J


In [15]:
# Reduce the data set size so that this runs faster
num_users_r = 4
num_movies_r = 5
num_features_r = 3

X_r = X[:num_movies_r, :num_features_r]
W_r = W[:num_users_r,  :num_features_r]
b_r = b[0, :num_users_r].reshape(1,-1)
Y_r = Y[:num_movies_r, :num_users_r]
R_r = R[:num_movies_r, :num_users_r]

J = cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 0)
print(f"Cost : {J:0.2f}")

Cost : 13.67


In [17]:
# Evaluate the cost function with rgularization
J = cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 1.5)
print(f"Cost (with regularization): {J:0.2f}")

Cost (with regularization): 28.09


In [19]:
def cofi_cost_func_v(X,W,b,Y,R, lambda_):
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5*tf.reduce_sum(j**2) + (lambda_/2)*(tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

In [20]:
# Evaluate cost function
J = cofi_cost_func_v(X_r, W_r, b_r, Y_r, R_r, 0);
print(f"Cost: {J:0.2f}")

# Evaluate cost function with regularization
J = cofi_cost_func_v(X_r, W_r, b_r, Y_r, R_r, 1.5);
print(f"Cost (with regularization): {J:0.2f}")

Cost: 13.67
Cost (with regularization): 28.09


In [26]:
movieList, movieList_df = load_Movie_List_pd()
my_ratings = np.zeros(num_movies)

# Ratings for the selected movie
my_ratings[2700] = 5
my_ratings[2609] = 2
my_ratings[929]  = 5   # Lord of the Rings: The Return of the King, The
my_ratings[246]  = 5   # Shrek (2001)
my_ratings[2716] = 3   # Inception
my_ratings[1150] = 5   # Incredibles, The (2004)
my_ratings[382]  = 2   # Amelie (Fabuleux destin d'Amélie Poulain, Le)
my_ratings[366]  = 5   # Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
my_ratings[622]  = 5   # Harry Potter and the Chamber of Secrets (2002)
my_ratings[988]  = 3   # Eternal Sunshine of the Spotless Mind (2004)
my_ratings[2925] = 1   # Louis Theroux: Law & Disorder (2008)
my_ratings[2937] = 1   # Nothing to Declare (Rien à déclarer)
my_ratings[793]  = 5   # Pirates of the Caribbean: The Curse of the Black Pearl (2003)

my_rated = [i for i in range(len(my_ratings)) if my_ratings[i]>0]

print('\n New user ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print(f'Rated {my_ratings[i]} for {movieList_df.loc[i, "title"]}')


 New user ratings:

Rated 5.0 for Shrek (2001)
Rated 5.0 for Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Rated 2.0 for Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Rated 5.0 for Harry Potter and the Chamber of Secrets (2002)
Rated 5.0 for Pirates of the Caribbean: The Curse of the Black Pearl (2003)
Rated 5.0 for Lord of the Rings: The Return of the King, The (2003)
Rated 3.0 for Eternal Sunshine of the Spotless Mind (2004)
Rated 5.0 for Incredibles, The (2004)
Rated 2.0 for Persuasion (2007)
Rated 5.0 for Toy Story 3 (2010)
Rated 3.0 for Inception (2010)
Rated 1.0 for Louis Theroux: Law & Disorder (2008)
Rated 1.0 for Nothing to Declare (Rien à déclarer) (2010)


In [27]:
Y, R = load_ratings_small()
Y = np.c_[my_ratings,Y]
R = np.c_[(my_ratings !=0).astype(int), R]

Ynorm, Ymean = normalizeRatings(Y,R)

In [28]:
num_movies, num_users = Y.shape
num_features = 100

tf.random.set_seed(1234)
W = tf.Variable(tf.random.normal((num_users, num_features), dtype= tf.float64), name = 'W')
X = tf.Variable(tf.random.normal((num_movies, num_features), dtype = tf.float64), name = 'X')
b = tf.Variable(tf.random.normal((1, num_users), dtype = tf.float64), name = 'b')

optimizer = keras.optimizers.Adam(learning_rate = 1e-1)

In [32]:
iterations = 200
lambda_ = 1
for iter in range(iterations):
    with tf.GradientTape() as tape:
        cost_value = cofi_cost_func_v(X,W, b, Ynorm, R, lambda_)
    grads = tape.gradient(cost_value, [X,W,b])
    optimizer.apply_gradients(zip(grads,[X,W,b]))

    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 1765.5
Training loss at iteration 20: 1771.4
Training loss at iteration 40: 1768.7
Training loss at iteration 60: 1766.1
Training loss at iteration 80: 1769.0
Training loss at iteration 100: 1766.1
Training loss at iteration 120: 1769.3
Training loss at iteration 140: 1767.3
Training loss at iteration 160: 1768.3
Training loss at iteration 180: 1769.4


In [33]:
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()
pm = p + Ymean

my_predictions = pm[:,0]
ix = tf.argsort(my_predictions, direction = 'DESCENDING')

for i in range(17):
    j = ix[i]
    if j not in my_rated:
        print(f"Predicting rating {my_predictions[j]:0.2f} for movie {movieList[j]}")

print("\n\n Original vs Predicted ratings:\n")
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print(f"Original {my_ratings[i]}, Predicted {my_predictions[i]:0.2f} for {movieList[i]}")

Predicting rating 4.73 for movie Colourful (Karafuru) (2010)
Predicting rating 4.58 for movie Battle Royale 2: Requiem (Batoru rowaiaru II: Chinkonka) (2003)
Predicting rating 4.58 for movie Into the Abyss (2011)
Predicting rating 4.58 for movie Eichmann (2007)
Predicting rating 4.57 for movie Delirium (2014)
Predicting rating 4.57 for movie Laggies (2014)
Predicting rating 4.57 for movie One I Love, The (2014)
Predicting rating 4.55 for movie Particle Fever (2013)
Predicting rating 4.54 for movie 'Salem's Lot (2004)
Predicting rating 4.53 for movie Kung Fu Panda: Secrets of the Masters (2011)


 Original vs Predicted ratings:

Original 5.0, Predicted 4.90 for Shrek (2001)
Original 5.0, Predicted 4.84 for Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Original 2.0, Predicted 2.19 for Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Original 5.0, Predicted 4.88 for Harry Potter and the Chamber of Secrets (2002)
Original 5.0, Predicted 

In [35]:
filter=(movieList_df["number of ratings"] > 20)
movieList_df["pred"] = my_predictions
movieList_df = movieList_df.reindex(columns = ["pred", "mean rating", "number of ratings", "title"])
movieList_df.loc[ix[:300]].loc[filter].sort_values("mean rating", ascending = False)

,pred,mean rating,number of ratings,title
2112,4.275778,4.238255,149,"Dark Knight, The (2008)"
155,4.241571,4.155914,93,Snatch (2000)
929,4.871031,4.118919,185,"Lord of the Rings: The Return of the King, The..."
2700,4.789260,4.109091,55,Toy Story 3 (2010)
393,4.408934,4.106061,198,"Lord of the Rings: The Fellowship of the Ring,..."
653,4.364353,4.021277,188,"Lord of the Rings: The Two Towers, The (2002)"
2804,4.162813,3.989362,47,Harry Potter and the Deathly Hallows: Part 1 (...
3283,4.024632,3.982143,28,Argo (2012)
773,4.363168,3.960993,141,Finding Nemo (2003)
2649,4.022647,3.943396,53,How to Train Your Dragon (2010)


In [36]:
movieList_df

,pred,mean rating,number of ratings,title
0,2.937348,3.400000,5,"Yards, The (2000)"
1,2.704054,3.250000,6,Next Friday (2000)
2,1.558169,2.000000,4,Supernova (2000)
3,1.553139,2.000000,4,Down to You (2000)
4,2.453352,2.672414,29,Scream 3 (2000)
...,...,...,...,...
4773,3.009142,3.500000,1,Jon Stewart Has Left the Building (2015)
4774,3.509135,4.000000,1,Black Butler: Book of the Atlantic (2017)
4775,3.011553,3.500000,1,No Game No Life: Zero (2017)
4776,3.009280,3.500000,1,Flint (2017)


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(movieList_df, *['pred'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(movieList_df, *['mean rating'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(movieList_df, *['number of ratings'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(movieList_df, *['pred'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(movieList_df, *['mean rating'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(movieList_df, *['number of ratings'], **{})
chart

import numpy as np
from google.colab import autoviz

def scatter_plots(df, colname_pairs, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * 6 * figscale, 6 * figscale))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha, ax=ax)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(movieList_df, *[[['pred', 'mean rating'], ['mean rating', 'number of ratings']]], **{})
chart